## 네이버 뉴스 기사 스크래핑

## 참고 사이트
    * https://velog.io/@choi46910/%EA%B8%B0%EC%82%AC-%EC%9B%B9-%EC%8A%A4%ED%81%AC%EB%9E%98%ED%95%91%ED%81%AC%EB%A1%A4%EB%A7%81%ED%95%98%EA%B8%B0-%EC%97%91%EC%85%80-%EC%A0%80%EC%9E%A5
    * https://haerong22.tistory.com/66

In [1]:
from bs4 import BeautifulSoup
from selenium import webdriver
import csv, sys
import pandas as pd

In [5]:
driver = webdriver.Chrome('chromedriver')

# 검색어 입력
search_name = input('검색어를 입력하세요 : ')

# url = 'https://search.naver.com/search.naver?sm=tab_hty.top&where=news&query={}&start={}'.format(search_name, str(page_num))
url = 'https://search.naver.com/search.naver?sm=tab_hty.top&where=news&query={}'.format(search_name)
driver.get(url)
req = driver.page_source
soup = BeautifulSoup(req, 'html.parser')

articles = soup.select('#main_pack > section.sc_new.sp_nnews._prs_nws > div > div.group_news > ul > li')
print(len(articles), '개의 기사가 검색됨.')

# 리스트로 구성된 articles에서 딕셔너리나 인덱스를 통해 참조.
titles = list()
urls = list()
presses = list()

# 스타트 앤드 넘버로 페이지 수 확인.
for article in articles:
    title = article.select_one('div.news_wrap.api_ani_send > div > a').text
    url = article.select_one('div.news_wrap.api_ani_send > div > a')['href']
    press = article.select_one('a.info.press').text.split(' ')[0].replace('언론사', '')
    # 문제
    date = article.select_one('a.info').text.split(' ')
    titles.append(title)
    urls.append(url)
    presses.append(press)
    print(date)
    break
sys.exit()
# 스캔 종료
driver.quit()

news_df = pd.DataFrame({'제목' : titles, '사이트' : urls, '언론사' : presses})
news_df.to_csv('네이버뉴스_{}.csv'.format(search_name), index=False, encoding='utf-8-sig')

검색어를 입력하세요 : ㅏ
10 개의 기사가 검색됨.
['조이뉴스24']


SystemExit: 

C:\Users\DGBDS\anaconda3\envs\check\lib\site-packages\IPython\core\interactiveshell.py:2886: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [53]:
pd.read_csv('네이버뉴스_DGB대구은행.csv')

,제목,사이트,언론사
0,"DGB대구은행, 삼성라이온즈·대구FC 특판 예·적금 출시",http://www.newsis.com/view/?id=NISX20220420_00...,뉴시스
1,"DGB대구은행, '외화 E-지갑’ 출시",http://www.breaknews.com/888424,브레이크뉴스
2,"DGB대구은행, 사회공헌활동 남구 지역민 '소망 도시락' 봉사",https://www.gukjenews.com/news/articleView.htm...,국제뉴스
3,환전은 쉽게…DGB대구은행 '외화 E-지갑' 출시,https://www.news1.kr/articles/?4652198,뉴스1
4,"대구은행, 남구 지역민 소망 도시락 전달 '훈훈'",https://www.sisa-news.com/news/article.html?no...,시사뉴스
5,1100억원에 나온 7400평 ‘은행 땅’…대도시에 쏠렸다,https://view.asiae.co.kr/article/2022042009201...,아시아경제
6,"금융노조 대구은행지부, 멘토리 야구단에 2천만 원 후원",https://www.nocutnews.co.kr/news/5742738,노컷뉴스
7,‘관중 물병 투척’ 홈구단 대구FC에 제재금,https://news.kbs.co.kr/news/view.do?ncd=544437...,KBS
8,'관중이 심판 향해 물병 투척' 대구FC에 제재금 300만원,https://www.yna.co.kr/view/AKR2022041813520000...,연합뉴스
9,"DGB대구은행, ICT·디지털 6급 경력직 채용",http://daily.hankooki.com/news/articleView.htm...,데일리한국


## 깃허브 소스

In [1]:
# -*- coding: utf-8 -*-
from bs4 import BeautifulSoup
from datetime import datetime
import requests
import pandas as pd
import re
'''''''''''''''''''''''''''''''''''''''''''''''''''''''''
< naver 뉴스 검색시 리스트 크롤링하는 프로그램 > _select사용
- 크롤링 해오는 것 : 링크,제목,신문사,날짜,내용요약본
- 날짜,내용요약본  -> 정제 작업 필요
- 리스트 -> 딕셔너리 -> df -> 엑셀로 저장 
'''''''''''''''''''''''''''''''''''''''''''''''''''''''''

#각 크롤링 결과 저장하기 위한 리스트 선언 
title_text=[]
link_text=[]
source_text=[]
date_text=[]
contents_text=[]
result={}

#엑셀로 저장하기 위한 변수

RESULT_PATH = 'C:/Users/DGBDS/Desktop/'  #결과 저장할 경로
now = datetime.now() #파일이름 현 시간으로 저장하기

#날짜 정제화 함수
def date_cleansing(test):
    try:
        #지난 뉴스
        #머니투데이  10면1단  2018.11.05.  네이버뉴스   보내기  
        pattern = '\d+.(\d+).(\d+).'  #정규표현식 
    
        r = re.compile(pattern)
        match = r.search(test).group(0)  # 2018.11.05.
        date_text.append(match)
        
    except AttributeError:
        #최근 뉴스
        #이데일리  1시간 전  네이버뉴스   보내기  
        pattern = '\w* (\d\w*)'     #정규표현식 
        
        r = re.compile(pattern)
        match = r.search(test).group(1)
        #print(match)
        date_text.append(match)


#내용 정제화 함수 
def contents_cleansing(contents):
    first_cleansing_contents = re.sub('<dl>.*?</a> </div> </dd> <dd>', '', 
                                      str(contents)).strip()  #앞에 필요없는 부분 제거
    second_cleansing_contents = re.sub('<ul class="relation_lst">.*?</dd>', '', 
                                       first_cleansing_contents).strip()#뒤에 필요없는 부분 제거 (새끼 기사)
    third_cleansing_contents = re.sub('<.+?>', '', second_cleansing_contents).strip()
    contents_text.append(third_cleansing_contents)
    #print(contents_text)
    

def crawler(maxpage,query,sort,s_date,e_date):

    s_from = s_date.replace(".","")
    e_to = e_date.replace(".","")
    page = 1  
    maxpage_t =(int(maxpage)-1)*10+1   # 11= 2페이지 21=3페이지 31=4페이지  ...81=9페이지 , 91=10페이지, 101=11페이지
    
    while page <= maxpage_t:
        url = "https://search.naver.com/search.naver?where=news&query=" + query + "&sort="+sort+"&ds=" + s_date + "&de=" + e_date + "&nso=so%3Ar%2Cp%3Afrom" + s_from + "to" + e_to + "%2Ca%3A&start=" + str(page)
        
        response = requests.get(url)
        html = response.text
 
        #뷰티풀소프의 인자값 지정
        soup = BeautifulSoup(html, 'html.parser')
 
        #<a>태그에서 제목과 링크주소 추출
        atags = soup.select('.news_tit')
        for atag in atags:
            title_text.append(atag.text)     #제목
            link_text.append(atag['href'])   #링크주소
            
        #신문사 추출
        source_lists = soup.select('.info_group > .press')
        for source_list in source_lists:
            source_text.append(source_list.text)    #신문사
        
        #날짜 추출 
        date_lists = soup.select('.info_group > span.info')
        for date_list in date_lists:
            # 1면 3단 같은 위치 제거
            if date_list.text.find("면") == -1:
                date_text.append(date_list.text)
        
        #본문요약본
        contents_lists = soup.select('.news_dsc')
        for contents_list in contents_lists:
            contents_cleansing(contents_list) #본문요약 정제화
        

        #모든 리스트 딕셔너리형태로 저장
        result= {"date" : date_text , "title":title_text ,  "source" : source_text ,"contents": contents_text ,"link":link_text }  
        print(page)
        
        df = pd.DataFrame(result)  #df로 변환
        page += 10
    
    
    # 새로 만들 파일이름 지정
    outputFileName = '%s-%s-%s  %s시 %s분 %s초 merging.xlsx' % (now.year, now.month, now.day, now.hour, now.minute, now.second)
    df.to_excel(RESULT_PATH+outputFileName,sheet_name='sheet1')
    
    

def main():
    info_main = input("="*50+"\n"+"입력 형식에 맞게 입력해주세요."+"\n"+" 시작하시려면 Enter를 눌러주세요."+"\n"+"="*50)
    
    maxpage = input("최대 크롤링할 페이지 수 입력하시오: ")  
    query = input("검색어 입력: ")  
    sort = input("뉴스 검색 방식 입력(관련도순=0  최신순=1  오래된순=2): ")    #관련도순=0  최신순=1  오래된순=2
    s_date = input("시작날짜 입력(2019.01.04):")  #2019.01.04
    e_date = input("끝날짜 입력(2019.01.05):")   #2019.01.05
    
    crawler(maxpage,query,sort,s_date,e_date) 
    
main()

입력 형식에 맞게 입력해주세요.
 시작하시려면 Enter를 눌러주세요.
최대 크롤링할 페이지 수 입력하시오: 1000
검색어 입력: DGB대구은행
뉴스 검색 방식 입력(관련도순=0  최신순=1  오래된순=2): 2
시작날짜 입력(2019.01.04):2021.08.01
끝날짜 입력(2019.01.05):2021.12.31
1
11
21
31
41
51
61
71
81
91
101
111
121
131
141
151
161
171
181
191
201
211
221
231
241
251
261
271
281
291
301
311
321
331
341
351
361
371
381
391
401
411
421
431
441
451
461
471
481
491
501
511
521
531
541
551
561
571
581
591
601
611
621
631
641
651
661
671
681
691
701
711
721
731
741
751
761
771
781
791
801
811
821
831
841
851
861
871
881
891
901
911
921
931
941
951
961
971
981
991
1001
1011
1021
1031
1041
1051
1061
1071
1081
1091
1101
1111
1121
1131
1141
1151
1161
1171
1181
1191
1201
1211
1221
1231
1241
1251
1261
1271
1281
1291
1301
1311
1321
1331
1341
1351
1361
1371
1381
1391
1401
1411
1421
1431
1441
1451
1461
1471
1481
1491
1501
1511
1521
1531
1541
1551
1561
1571
1581
1591
1601
1611
1621
1631
1641
1651
1661
1671
1681
1691
1701
1711
1721
1731
1741
1751
1761
1771
1781
1791
1801
1811
1821
1831
1841
1851
